# Open-Neural-APC as Tensorflow 2 implementation

In [1]:
# install and update required packages
!pip install --upgrade pip -q
!pip install --upgrade -r requirements.txt -q

# just for the video output/investigation (not necessary for training/testing)
!pip install --upgrade -r optionals.txt -q
# this is also optional, since it is needed for opencv
!apt-get -qq update && apt-get -qq install -y libsm6 libxext6 libxrender1 libfontconfig1

In [2]:
from utils import loadConfig, allow_growth
from tqdm import tqdm
# read the config file
# it includes more or less all hyperparameter used in the model and preprocessing/training step
data_parameter, model_parameter, training_parameter = loadConfig()
# Since I saved the data as "uint8" and the sensor is usually placed at a height of 2 meters
#    the resolution should be just below 1cm
#    but since the noise frames at the end of most sequences produce larger values, this might not be the case
# The "accuracy error niveau" is the absolutely permissible difference so that a prediction 
#    on the label is still counted as correct (remember: it's a regression task)
# The "jump input frames" parameter indicates which frames are used from the original sequence
# The original sequences are at about 40 FPS. The model is trained with just 10 FPS
# The "pretrain" parameter is not used so far but could be utilized with a pretraining of the input layer
# The "safe steps" parameter was used for the non-jupyter version to safe the model every 5 epochs

# enable GPU memory growth 
allow_growth()

{
  "data parameter": {
    "data directory": "./data/",
    "input scaling factor": 255,
    "labels dtype": "uint32",
    "sequence dtype": "uint8",
    "training label": "train.csv",
    "validation label": "valid.csv"
  },
  "model parameter": {
    "input dimensions": [
      20,
      25
    ],
    "lstm depth": 5,
    "lstm width": 50,
    "output dimensions": 2
  },
  "training parameter": {
    "accuracy error niveau": 0.5,
    "aux scale": 3,
    "batch size": 16,
    "calculation dtype": "float32",
    "calculation epsilon": 1e-07,
    "dropout rate": 0.2,
    "epochs": 10000,
    "jump input frames": 4,
    "learning rate": 0.001,
    "maximum concatenation": 6,
    "minimum concatenation": 4,
    "optimizer parameter": [
      0.9,
      0.999,
      1e-07
    ],
    "pretrain": false,
    "restrict dataset size": 1500,
    "safe steps": 5
  }
}


In [3]:
# enable effcient data processing
sequence_dtype = data_parameter["sequence dtype"]
labels_dtype = data_parameter["labels dtype"]
calculation_dtype = training_parameter["calculation dtype"]
calculation_epsilon = training_parameter["calculation epsilon"]

# enable single/half/double precision
import tensorflow.keras.backend as K
K.set_floatx(calculation_dtype)
K.set_epsilon(calculation_epsilon)

import numpy as np
# scale saved input into a normed range 
input_scaling_factor = np.asarray(data_parameter["input scaling factor"],dtype=calculation_dtype)

In [4]:
from napc import NeuralAPC
# This is the pure model. It's almost the same as in my bachelors thesis
# The only difference is an additional loss (auxiliary loss) and a dropout layer after the first dense layer
# Elsewise it's the same architecture, same optimizer, same 'main' loss
napc = NeuralAPC(model_parameter,training_parameter)
napc.compile()
napc.save()

# copy config into model folder
import shutil
config_path = shutil.copy2('config.json', napc.model_path)
print(f'Config saved under {config_path}')

Config saved under ./models/2020-06-04_17:03:46.847015/config.json


In [5]:
import os
import os.path as path
mode = "training"
data = path.join(data_parameter['data directory'], f'{mode}.dat')
data_lengths = path.join(data_parameter['data directory'], f'{mode}_meta.dat')

# This one solves the problem for all of you, which just click 'Run all' and expect it to work
# Since the training data is not uploaded 'on purpose', the code would usually break for most people
# If the file does not exists, the code immediately jumps to the point, where it loads the 'predefined' model and evaluates it
# Elsewise it trains the model and evalutes the newly trained
data_exists = os.path.isfile(data)

sequence_list = labels_list = None

if data_exists:
    from data_loader import readData
    # due to the expertise of github.com/xor2k, I switched from CSV to memory-mapped files.
    # This reduces the loading/mapping time by a lot.
    sequence_list, labels_list = readData(model_parameter,training_parameter,data,data_lengths,sequence_dtype,labels_dtype)
    # Similar to my bachelors thesis I used just about 1500 sequences (Not necessarily the same ones/ haven't checked it)
    sequence_list, labels_list = sequence_list[:training_parameter["restrict dataset size"]], labels_list[:training_parameter["restrict dataset size"]]

In [6]:
from data_processing import Preprocess
preprocessor = None
if data_exists:
    # this is class which preprocesses the data every epoch
    # it creates the necessary labels/bounds and augments the data
    preprocessor = Preprocess(sequence_list,labels_list,input_scaling_factor,training_parameter,calculation_dtype)

In [7]:
# train
if data_exists:
    
    from matplotlib import pyplot as plt
    %matplotlib inline

    import time as t
    train_start = t.time()

    for epoch in tqdm(range(napc.epoch,training_parameter['epochs']),desc='Training progress'):
        start = t.time()

        X,Y = preprocessor.prepareEpoch()
        prep_end = t.time()

        for x,y in list(zip(X,Y)):
            metrics = napc.model.train_on_batch(x,y,reset_metrics=False)

        if epoch%100==0:
            print(','.join( np.vstack([['epoch'],[epoch]]).T.flatten() )+','+\
                  ','.join( np.vstack([['t_total','t_prep','t_calc'],\
                            [round(t.time()-train_start,2),round(prep_end-start,2),\
                             round(t.time()-prep_end,2)]]).T.flatten() )+','+\
                  ','.join( np.vstack([napc.model.metrics_names,np.round(metrics,5)]).T.flatten() ))

        napc.model.reset_metrics()
        napc.epoch += 1

        if epoch%1000==0:

            sample = 0
            x = x[sample:sample+1,:,:]
            y = y[sample:sample+1,:,:]

            prediction = napc.model.predict_on_batch(x)
            error = napc.loss_function(y[:,:,:2],y[:,:,2:4],prediction)

            # prepare plot
            num_plots = 2
            fig, ax = plt.subplots(1,num_plots,figsize=(14,3), dpi=80)
            fig.suptitle('Predictions in/out')

            for idx in range(num_plots):
                ax[idx].plot(y[0,:,idx],label='max')
                ax[idx].plot(y[0,:,idx+2],label='min')
                ax[idx].plot(y[0,:,4],label='end')
                ax[idx].plot(prediction[0,:,idx],label='prediction')
                ax[idx].plot(error[0,:,idx],label='error')
                ax[idx].legend()

            plt.show(fig)
    napc.save()
    # the output of this cell are metrics every 10th epoch and a plot of the counting behaviour every 100th epoch
    # the training took about 10 hours on a 2080TI while occupying less than 2GB VRAM and 80% GPU-Util.

In [8]:
# For all of you which don't have the training data and just want to execute it
if not data_exists:
    # Loading the included model (it has no subdirectory)
    napc.loadModel(10000,'models/')
    # The model_path of the model is not 'models/', but the previously created subdirectory
    # You could now train it further/save it/ etc.

In [9]:
# produce videos on all validation sequences
import os.path as path
mode = "validation"
data = path.join(data_parameter['data directory'], f'{mode}.dat')
data_lengths = path.join(data_parameter['data directory'], f'{mode}_meta.dat')

# copy dict from training and modify the concatenation and batch size
validation_parameter = training_parameter.copy()
validation_parameter["minimum concatenation"] = 1
validation_parameter["maximum concatenation"] = 1
validation_parameter["batch size"]=1

# read the validation data
from data_loader import readData
sequence_list, labels_list = readData(model_parameter,validation_parameter,data,data_lengths,sequence_dtype,labels_dtype)

# process them (i need the bounds in y for the accuracy and the videos)
from data_processing import Preprocess
preprocessor = Preprocess(sequence_list,labels_list,input_scaling_factor,validation_parameter,calculation_dtype)
X,Y = preprocessor.prepareEpoch(training=False)

# create my videos
from optional_features import createVideo
accuracy = []
for idx,x,y in list(zip(range(len(X)),X,Y)):
    prediction = napc.model.predict_on_batch(x)
    createVideo(napc.epoch,idx,x,K.eval(prediction),y[:,:,0:2],y[:,:,2:4])
    accuracy += [K.eval(napc.accuracy(y,prediction))]

Started reading files: 17:03:50 2020-06-04
Finished reading 15 sequences. Took 0.000000 seconds.


Video progress: 534it [02:32,  3.51it/s]
Video progress: 141it [00:33,  4.27it/s]
Video progress: 146it [00:34,  4.24it/s]
Video progress: 504it [02:10,  3.86it/s]


Video progress: 131it [00:29,  4.49it/s]
Video progress: 146it [00:34,  4.28it/s]


Video progress: 147it [00:34,  4.26it/s]


Video progress: 116it [00:24,  4.67it/s]


...

Video progress: 630it [03:32,  2.96it/s]


Video progress: 151it [00:36,  4.19it/s]


Video progress: 223it [00:48,  4.58it/s]


Video progress: 137it [00:32,  4.23it/s]


Video progress: 293it [01:14,  3.91it/s]
Video progress: 146it [00:34,  4.18it/s]


In [10]:
# Since I'm not allowed to upload more sequences and I don't want to publish a perfect model
# (therefore, I haven't tested this one) the accuracy is just an approximation of the true capabilities
# The 'validation' data in this case is in fact a test set (last epoch was chosen without selection)
# In practice someone would use k-Fold-Crossvalidation and would reason about the average performance

# So let's have a look how well the model does...
print(f'{mode} accuracy: {100*np.mean(accuracy)} %')
# Not bad, it's even above 96%!

validation accuracy: 96.66666388511658 %
